# Domain Data Preprocessing

In [519]:
# Libraries
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import numpy as np
from pathlib import Path
import statsmodels.api as sm
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from fuzzywuzzy import process
from collections import Counter

In [520]:
# Load data
domain_df = pd.read_csv("/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/datasets/property/domain/vic_rentals_all.csv")
print(domain_df.shape)
domain_df.head()


(12717, 30)


,listing_id,suburb,postcode,weekly_rent,bond,available_date,date_listed,days_listed,bedrooms,bathrooms,...,floorplans_count,virtual_tour,primary_type,secondary_type,agency,agency_id,agent_names,structured_features,url,land_area
0,16782629,SOUTH KINGSVILLE,3015,460.0,1994.0,"Tuesday, 02 September 2025",2025-08-13,27.0,2.0,1.0,...,0.0,False,Apartment,Apartment / Unit / Flat,Lease A Property,21230.0,Marc Angelone,"Built in wardrobes, Secure Parking, Bath, Heat...",https://www.domain.com.au/3-53-greene-street-s...,NaN
1,17471867,SOUTH KINGSVILLE,3015,400.0,1738.0,"Thursday, 27 March 2025",2025-03-06,187.0,2.0,1.0,...,0.0,False,Apartment,Apartment / Unit / Flat,Village Real Estate,20880.0,Trudie Thobe,"Internal Laundry, Pets Allowed, Heating",https://www.domain.com.au/1-3-new-street-south...,NaN
2,17721851,SOUTH KINGSVILLE,3015,795.0,3454.0,"Monday, 15 September 2025",2025-08-19,21.0,3.0,2.0,...,1.0,False,Townhouse/Villa,Townhouse,Jas Stephens Real Estate,22.0,Jesseigh Stella,"Internal Laundry, Balcony / Deck, Floorboards,...",https://www.domain.com.au/19-92-new-street-sou...,NaN
3,17725855,SOUTH KINGSVILLE,3015,675.0,2933.0,"Wednesday, 20 August 2025",2025-08-21,19.0,3.0,1.0,...,0.0,False,Townhouse/Villa,Townhouse,Jellis Craig Inner West,19574.0,Zac Keltie,"Air conditioning, Built in wardrobes, Balcony ...",https://www.domain.com.au/3-14-saltley-street-...,NaN
4,17745057,SOUTH KINGSVILLE,3015,450.0,1955.0,"Tuesday, 02 September 2025",2025-09-03,6.0,2.0,1.0,...,0.0,False,Apartment,Apartment / Unit / Flat,Belle Property Albert Park,3702.0,"William Brydon Waldren, Shar Claridge","Built in wardrobes, Heating",https://www.domain.com.au/4-2b-saltley-street-...,NaN


### Match Domain Data to SA2

In [521]:
# Load data
sa2_df = gpd.read_file("/Users/tasneemzulaiqa/Documents/GitHub/project-2-group-real-estate-industry-project-7-2025/datasets/district_shape/SA2_GDA2020_SHAPEFILE/SA2_2021_AUST_GDA2020.shp")
sa2_df.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.218 -35.366..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.1997 -35.35..."


In [522]:
# Convert property lat/lon to geometry points
gdf_points = gpd.GeoDataFrame(domain_df, geometry=gpd.points_from_xy(domain_df['lon'], domain_df['lat']), crs="EPSG:4326" )
sa2_df = sa2_df.to_crs(gdf_points.crs)

# Join 
domain_sa2 = gpd.sjoin(gdf_points, sa2_df, how="left", predicate="within")

In [523]:
# Check matching quality
total_props = len(domain_sa2)
matched_props = domain_sa2['SA2_CODE21'].notna().sum()
matching_rate = matched_props / total_props * 100

print(f"Matched {matched_props} out of {total_props} properties ({matching_rate:.2f}%)")

Matched 12713 out of 12717 properties (99.97%)


In [524]:
# suburbs that failed to match
unmatched_suburbs = domain_sa2.loc[domain_sa2['SA2_CODE21'].isna(), 'suburb'].unique()
print("unmatched suburbs:", unmatched_suburbs[:20]) 


unmatched suburbs: ['Rowville' 'Irymple' 'Coimadai']


In [525]:
domain_sa2.rename(columns={'SA2_CODE21': 'sa2_code','SA2_NAME21': 'sa2_name'},inplace=True)
domain_sa2.columns

Index(['listing_id', 'suburb', 'postcode', 'weekly_rent', 'bond',
       'available_date', 'date_listed', 'days_listed', 'bedrooms', 'bathrooms',
       'carspaces', 'property_type', 'address', 'lat', 'lon', 'scraped_date',
       'domain_page_id', 'property_id', 'photo_count', 'video_count',
       'floorplans_count', 'virtual_tour', 'primary_type', 'secondary_type',
       'agency', 'agency_id', 'agent_names', 'structured_features', 'url',
       'land_area', 'geometry', 'index_right', 'sa2_code', 'sa2_name',
       'CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21', 'SA3_NAME21', 'SA4_CODE21',
       'SA4_NAME21', 'GCC_CODE21', 'GCC_NAME21', 'STE_CODE21', 'STE_NAME21',
       'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21', 'LOCI_URI21'],
      dtype='object')

In [526]:
keep_cols = ['sa2_code', 'sa2_name','suburb','postcode', 'weekly_rent', 'bond', 'address', 'lat', 'lon', 'bedrooms', 'bathrooms', 'carspaces', 'property_type', 'land_area', 'structured_features', 'date_listed']
domain = domain_sa2[keep_cols].copy()
domain.head()

,sa2_code,sa2_name,suburb,postcode,weekly_rent,bond,address,lat,lon,bedrooms,bathrooms,carspaces,property_type,land_area,structured_features,date_listed
0,213021344,Newport,SOUTH KINGSVILLE,3015,460.0,1994.0,3/53 Greene Street,-37.830982,144.87091,2.0,1.0,2.0,Apartment / Unit / Flat,NaN,"Built in wardrobes, Secure Parking, Bath, Heat...",2025-08-13
1,213021344,Newport,SOUTH KINGSVILLE,3015,400.0,1738.0,1/3 New Street,-37.826218,144.86755,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"Internal Laundry, Pets Allowed, Heating",2025-03-06
2,213021343,Altona North,SOUTH KINGSVILLE,3015,795.0,3454.0,19/92 New Street,-37.831226,144.86632,3.0,2.0,3.0,Townhouse,NaN,"Internal Laundry, Balcony / Deck, Floorboards,...",2025-08-19
3,213021344,Newport,SOUTH KINGSVILLE,3015,675.0,2933.0,3/14 Saltley Street,-37.827423,144.86768,3.0,1.0,2.0,Townhouse,NaN,"Air conditioning, Built in wardrobes, Balcony ...",2025-08-21
4,213021344,Newport,SOUTH KINGSVILLE,3015,450.0,1955.0,4/2B Saltley Street,-37.826270,144.86790,2.0,1.0,NaN,Apartment / Unit / Flat,NaN,"Built in wardrobes, Heating",2025-09-03


### Turn 'structured_features' Column to Independent Columns

In [527]:
# Split into lists
feature_lists = domain['structured_features'].dropna().str.split(', ')

# Flatten
all_features = [feature for sublist in feature_lists for feature in sublist]

# Count frequencies
feature_counts = Counter(all_features)

# Convert to DataFrame
feature_df = pd.DataFrame(feature_counts.items(), columns=['Feature', 'Count'])
feature_df = feature_df.sort_values(by='Count', ascending=False).reset_index(drop=True)
feature_df.head(30)


,Feature,Count
0,Built in wardrobes,6690
1,Heating,5710
2,Dishwasher,5456
3,Secure Parking,4966
4,Internal Laundry,3993
5,Air conditioning,3796
6,Balcony / Deck,2509
7,Gas,2421
8,Floorboards,1904
9,Bath,1881


In [528]:
# save to inspect 
feature_df.to_csv('property_structured_features.csv', index=False)

### Fuzzy Match Features to Group Similar Features Together

In [529]:
canonical_features = ['balcony', 'car parking', 'heating', 'air conditioning', 'builtin wardrobes', 'laundry', 'swimming pool', 'ensuite',
    'bathroom features', 'dishwasher', 'garden', 'gym', 'security', 'pets allowed', 'gas', 'washing machine', 'alarm', 'intercom', 'internet']

# Find best match from canonical_features and accept if similar enough
def map_to_canonical(feature_string):
    feature = feature_string.lower().strip()
    best_match, score = process.extractOne(feature, canonical_features)
    if score >= 70:
        return best_match
    else:
        return feature  


In [530]:
def normalize_list(feature_list):
    
    if not isinstance(feature_list, list):
        return []  
    return list(set(map(map_to_canonical, feature_list)))

# Apply function
domain['normalized_features'] = (domain['structured_features'].fillna('').str.split(',').apply(normalize_list))

In [531]:
domain.head()

,sa2_code,sa2_name,suburb,postcode,weekly_rent,bond,address,lat,lon,bedrooms,bathrooms,carspaces,property_type,land_area,structured_features,date_listed,normalized_features
0,213021344,Newport,SOUTH KINGSVILLE,3015,460.0,1994.0,3/53 Greene Street,-37.830982,144.87091,2.0,1.0,2.0,Apartment / Unit / Flat,NaN,"Built in wardrobes, Secure Parking, Bath, Heat...",2025-08-13,"[builtin wardrobes, pets allowed, car parking,..."
1,213021344,Newport,SOUTH KINGSVILLE,3015,400.0,1738.0,1/3 New Street,-37.826218,144.86755,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"Internal Laundry, Pets Allowed, Heating",2025-03-06,"[laundry, pets allowed, heating]"
2,213021343,Altona North,SOUTH KINGSVILLE,3015,795.0,3454.0,19/92 New Street,-37.831226,144.86632,3.0,2.0,3.0,Townhouse,NaN,"Internal Laundry, Balcony / Deck, Floorboards,...",2025-08-19,"[builtin wardrobes, air conditioning, ensuite,..."
3,213021344,Newport,SOUTH KINGSVILLE,3015,675.0,2933.0,3/14 Saltley Street,-37.827423,144.86768,3.0,1.0,2.0,Townhouse,NaN,"Air conditioning, Built in wardrobes, Balcony ...",2025-08-21,"[builtin wardrobes, air conditioning, car park..."
4,213021344,Newport,SOUTH KINGSVILLE,3015,450.0,1955.0,4/2B Saltley Street,-37.826270,144.86790,2.0,1.0,NaN,Apartment / Unit / Flat,NaN,"Built in wardrobes, Heating",2025-09-03,"[builtin wardrobes, heating]"


In [532]:
# Check results
feature_lists = domain['normalized_features'].apply(lambda x: x if isinstance(x, list) else [])
all_normalized_features = [feature for sublist in feature_lists for feature in sublist]
normalized_feature_counts = Counter(all_normalized_features)
normalized_feature_df = pd.DataFrame(normalized_feature_counts.items(), columns=['Feature', 'Count'])
normalized_feature_df = normalized_feature_df.sort_values(by='Count', ascending=False).reset_index(drop=True)
normalized_feature_df.head(30)


,Feature,Count
0,heating,7693
1,builtin wardrobes,6691
2,dishwasher,5458
3,car parking,5269
4,air conditioning,4562
5,laundry,4158
6,balcony,3093
7,gas,2732
8,bathroom features,1918
9,floorboards,1904


In [533]:
normalized_feature_df.to_csv('normalized_property_structured_features.csv', index=False)

In [534]:
# Look at mapping (save to csv to check)
unique_features = pd.DataFrame({'Original': sorted(set(all_features))})
unique_features['Mapped'] = unique_features['Original'].apply(map_to_canonical)
unique_features.to_csv('feature_mapping_check.csv', index=False)

In [535]:
# Extract only some features and turn into independent columns
chosen = ['balcony', 'car parking', 'heating', 'air conditioning', 'built in wardrobes', 'laundry', 'swimming pool', 'ensuite', 'dishwasher',
    'garden', 'gym', 'security', 'pets allowed', 'gas', 'washing machine', 'alarm', 'intercom']

domain_expanded = domain.copy()

# Fill NaN with empty string
domain_expanded['normalized_features'] = domain_expanded['normalized_features'].fillna('')

# create a column for each chosen feature
for feature in chosen:
    domain_expanded[feature] = domain_expanded['normalized_features'].apply(lambda x: 1 if feature in x else 0)

# Drop original column 
domain_expanded = domain_expanded.drop(columns=['structured_features'])
domain_expanded[chosen].head()

,balcony,car parking,heating,air conditioning,built in wardrobes,laundry,swimming pool,ensuite,dishwasher,garden,gym,security,pets allowed,gas,washing machine,alarm,intercom
0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,1,0,1,1,0,1,0,1,1,0,0,0,1,1,0,0,0
3,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [536]:
# merge similar features
merge_dict = {'security_system': ['alarm', 'security'],'washing_machine': ['laundry', 'washing machine']}

for new_feat, old_feats in merge_dict.items():
    domain_expanded[new_feat] = domain_expanded['normalized_features'].apply(
        lambda x: any(feat in x for feat in old_feats) if isinstance(x, list) else False)

In [537]:
# remove unneeded columns and rename for consistency
for old_feats in merge_dict.values():
    for feat in old_feats:
        if feat in domain_expanded.columns:
            domain_expanded.drop(columns=feat, inplace=True)
            
domain_expanded = domain_expanded.drop(columns=['normalized_features'])
domain_expanded['security_system'] = domain_expanded['security_system'].astype(int)
domain_expanded['washing_machine'] = domain_expanded['washing_machine'].astype(int)
domain_expanded.columns = domain_expanded.columns.str.lower().str.replace(' ', '_')
domain_expanded.columns

Index(['sa2_code', 'sa2_name', 'suburb', 'postcode', 'weekly_rent', 'bond',
       'address', 'lat', 'lon', 'bedrooms', 'bathrooms', 'carspaces',
       'property_type', 'land_area', 'date_listed', 'balcony', 'car_parking',
       'heating', 'air_conditioning', 'built_in_wardrobes', 'swimming_pool',
       'ensuite', 'dishwasher', 'garden', 'gym', 'pets_allowed', 'gas',
       'intercom', 'security_system', 'washing_machine'],
      dtype='object')

## Data Cleaning

In [538]:
print(domain_expanded.isnull().sum())
print(domain_expanded.shape)

sa2_code                  4
sa2_name                  4
suburb                    0
postcode                  0
weekly_rent             279
bond                    782
address                 104
lat                       4
lon                       4
bedrooms                125
bathrooms                51
carspaces              1803
property_type             0
land_area             12715
date_listed               4
balcony                   0
car_parking               0
heating                   0
air_conditioning          0
built_in_wardrobes        0
swimming_pool             0
ensuite                   0
dishwasher                0
garden                    0
gym                       0
pets_allowed              0
gas                       0
intercom                  0
security_system           0
washing_machine           0
dtype: int64
(12717, 30)


In [539]:
# Obvious fixes
domain_expanded = domain_expanded.dropna(subset=['sa2_code', 'sa2_name'])
domain_expanded = domain_expanded.drop(columns=['land_area', 'date_listed'])

### Weekly Rent and Bond

In [540]:
domain_expanded[['weekly_rent', 'bond']].describe()

,weekly_rent,bond
count,12434.000000,11932.000000
mean,764.777626,2787.329702
std,9430.235319,2684.197655
min,0.000000,50.000000
25%,485.000000,2086.000000
50%,560.000000,2433.000000
75%,690.000000,2955.000000
max,808500.000000,212917.000000


In [541]:
# Check rent to bond ratio to see relationship
valid = domain_expanded.dropna(subset=['weekly_rent', 'bond'])
valid = valid[valid['weekly_rent'] > 0]
valid['bond_ratio'] = valid['bond'] / valid['weekly_rent']
print(valid['bond_ratio'].describe())
print(valid['bond_ratio'].round().value_counts())

count    11671.000000
mean         4.407425
std          4.006144
min          0.003224
25%          4.344444
50%          4.345122
75%          4.345652
max        434.524490
Name: bond_ratio, dtype: float64
bond_ratio
4.0      11050
6.0        269
5.0        190
7.0         48
2.0         38
1.0         37
3.0         21
9.0          5
0.0          5
8.0          3
11.0         1
14.0         1
435.0        1
12.0         1
13.0         1
Name: count, dtype: int64


Note: use rentx4 to compute bond and /4 for rent. also check max bond ratio (looks like error)

In [542]:
# Replace 0 with NaN 
domain_expanded['weekly_rent'] = domain_expanded['weekly_rent'].replace(0, np.nan)
domain_expanded['bond'] = domain_expanded['bond'].replace(0, np.nan)

# Handle nan values for bond and weekly rent using our logic
domain_expanded.loc[domain_expanded['weekly_rent'].isna() & domain_expanded['bond'].notna(),'weekly_rent'] = domain_expanded['bond'] / 4
domain_expanded.loc[domain_expanded['bond'].isna() & domain_expanded['weekly_rent'].notna(),'bond'] = domain_expanded['weekly_rent'] * 4

In [543]:
# Outlier Check
low_threshold = 50      
high_threshold = 10000
outliers = domain_expanded[(domain_expanded['weekly_rent'] < low_threshold) |
                           (domain_expanded['weekly_rent'] > high_threshold)]
outliers[['weekly_rent', 'bond', 'address', 'suburb', 'property_type']]

,weekly_rent,bond,address,suburb,property_type
282,33.00,132.0,14-16 Bubb Street,MOE,House
394,30.00,200.0,58 Saleyards Road,BENALLA,House
395,30.00,200.0,40 Gay Street,BENALLA,House
414,43.25,173.0,21 Hannah Street,BENALLA,House
1929,315000.00,1365.0,1/28 Elgin Street,MORWELL,Apartment / Unit / Flat
2586,808500.00,2607.0,83 Golf Links Road,BERWICK,House
3395,27.50,110.0,47 Chambers Street,MYRTLEFORD,House
3396,23.75,95.0,14 Jubilee Street,MYRTLEFORD,House
3397,33.00,132.0,8 Mcgeehan Cres,MYRTLEFORD,Apartment / Unit / Flat
3641,595000.00,2578.0,4 Wilfred Street,ROSEBUD,House


In [544]:
# get outliers
mask = domain_expanded['weekly_rent'] > 10000

# Fix weekly_rent by recomputing from bond
domain_expanded.loc[mask, 'weekly_rent'] = domain_expanded.loc[mask, 'bond'] / 4
domain_expanded.loc[mask, ['weekly_rent', 'bond', 'address', 'suburb']]

,weekly_rent,bond,address,suburb
1929,341.25,1365.0,1/28 Elgin Street,MORWELL
2586,651.75,2607.0,83 Golf Links Road,BERWICK
3641,644.50,2578.0,4 Wilfred Street,ROSEBUD
11193,27157.50,108630.0,HUG11-12/847 Whitehorse Road,BOX HILL


In [545]:
domain_expanded[['weekly_rent', 'bond']].describe()

,weekly_rent,bond
count,12678.000000,12678.000000
mean,629.606957,2781.054583
std,382.729761,2627.539643
min,12.500000,50.000000
25%,485.000000,2086.000000
50%,560.000000,2433.000000
75%,690.000000,2955.000000
max,27157.500000,212917.000000


In [546]:
print(domain_expanded['property_type'].value_counts())

property_type
House                            6586
Apartment / Unit / Flat          4516
Townhouse                        1277
Studio                            209
New Apartments / Off the Plan      45
Villa                              28
Acreage / Semi-Rural               13
Semi-Detached                      10
New House & Land                    8
Car Space                           7
Duplex                              3
Block of Units                      3
Terrace                             3
Farm                                2
Unknown                             2
Vacant land                         1
Name: count, dtype: int64


In [547]:
# Filter to only property types we want
residential_types = [
    "House", "Apartment / Unit / Flat", "Townhouse", "Villa",
    "Semi-Detached", "Terrace", "Duplex", "Acreage / Semi-Rural",
    "New Apartments / Off the Plan", "New House & Land", "Studio"
]

domain_expanded = domain_expanded[domain_expanded['property_type'].isin(residential_types)]

# Apply rent minimum threshold 
domain_expanded = domain_expanded[domain_expanded['weekly_rent'] >= 100]

print("After filtering:", domain_expanded.shape)
print(domain_expanded['weekly_rent'].describe())

After filtering: (12618, 28)
count    12618.000000
mean       632.088445
std        381.736426
min        100.000000
25%        490.000000
50%        565.000000
75%        690.000000
max      27157.500000
Name: weekly_rent, dtype: float64


### Bedrooms, Bathrooms and Carspaces

In [548]:
# see missing by property type
missing_by_type = domain_expanded.groupby('property_type')[['bedrooms', 'bathrooms', 'carspaces']] \
                                 .apply(lambda x: x.isna().sum())
print(missing_by_type)

                               bedrooms  bathrooms  carspaces
property_type                                                
Acreage / Semi-Rural                  2          2          4
Apartment / Unit / Flat              28          4       1016
Duplex                                0          0          0
House                                 5          4        517
New Apartments / Off the Plan         0          0         40
New House & Land                      0          0          1
Semi-Detached                         0          0          3
Studio                               48          0        154
Terrace                               0          0          2
Townhouse                             0          0         30
Villa                                 0          0          1


In [549]:
# see median by property type
stats_by_type = domain_expanded.groupby('property_type')[['bedrooms', 'bathrooms', 'carspaces']] \
                               .median()
print(stats_by_type)

                               bedrooms  bathrooms  carspaces
property_type                                                
Acreage / Semi-Rural                4.0        2.0        6.0
Apartment / Unit / Flat             2.0        1.0        1.0
Duplex                              3.0        1.0        1.0
House                               3.0        2.0        2.0
New Apartments / Off the Plan       1.0        1.0        1.0
New House & Land                    3.0        1.5        2.0
Semi-Detached                       2.0        1.0        2.0
Studio                              1.0        1.0        1.0
Terrace                             2.0        1.0        2.0
Townhouse                           3.0        2.0        2.0
Villa                               2.0        1.0        2.0


In [550]:
# If car_parking == 0, set carspaces to 0
domain_expanded.loc[(domain_expanded['car_parking'] == 0) & (domain_expanded['carspaces'].isna()), 'carspaces'] = 0

# Impute with median for the rest 
for col in ['bedrooms', 'bathrooms', 'carspaces']:
    domain_expanded[col] = domain_expanded.groupby('property_type')[col].transform(
        lambda x: x.fillna(x.median())
    )

In [551]:
domain_expanded.isnull().sum()

sa2_code               0
sa2_name               0
suburb                 0
postcode               0
weekly_rent            0
bond                   0
address               92
lat                    0
lon                    0
bedrooms               0
bathrooms              0
carspaces              0
property_type          0
balcony                0
car_parking            0
heating                0
air_conditioning       0
built_in_wardrobes     0
swimming_pool          0
ensuite                0
dishwasher             0
garden                 0
gym                    0
pets_allowed           0
gas                    0
intercom               0
security_system        0
washing_machine        0
dtype: int64

Note: we are ignoring address nulls for now (not sure if we need it or not)

In [563]:
print(domain_expanded.shape)
domain_expanded.describe()

(12618, 29)


,postcode,weekly_rent,bond,lat,lon,bedrooms,bathrooms,carspaces,balcony,car_parking,...,ensuite,dishwasher,garden,gym,pets_allowed,gas,intercom,security_system,washing_machine,median_weekly_rent_sa2
count,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,...,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000,12618.000000
mean,3236.588049,632.088445,2791.798859,-37.782531,144.966711,2.729513,1.588841,1.456649,0.244650,0.416152,...,0.141227,0.431764,0.067206,0.060548,0.082739,0.216199,0.129339,0.048423,0.329371,596.493561
std,279.876662,381.736426,2628.070359,0.387144,0.554406,1.162402,0.635962,1.051152,0.429897,0.492939,...,0.348269,0.495342,0.250388,0.238510,0.275498,0.411668,0.335588,0.214667,0.470003,127.950255
min,3000.000000,100.000000,200.000000,-38.829080,141.000550,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,230.000000
25%,3044.000000,490.000000,2086.000000,-37.905155,144.869400,2.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,525.000000
50%,3138.000000,565.000000,2433.000000,-37.823923,144.982790,3.000000,2.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,580.000000
75%,3220.000000,690.000000,2955.000000,-37.751548,145.110210,4.000000,2.000000,2.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,650.000000
max,3996.000000,27157.500000,212917.000000,-34.166810,149.755920,50.000000,12.000000,22.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1600.000000


In [ ]:
# convert data type for consistency
domain_expanded['bedrooms'] = domain_expanded['bedrooms'].astype(int)
domain_expanded['bathrooms'] = domain_expanded['bathrooms'].astype(int)
domain_expanded['carspaces'] = domain_expanded['carspaces'].astype(int)

In [ ]:
# add median rent (i dont think we need this doe)
domain_expanded['median_weekly_rent_sa2'] = domain_expanded.groupby('sa2_code')['weekly_rent'].transform('median')

In [560]:
domain_expanded.to_csv('../../datasets/raw/cleaned/domain_cleaned.csv', index=False)